In [27]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-core langchain_openai python-dotenv langsmith pydantic spotipy

In [2]:
%pip install --quiet -U jupyterlab-lsp
%pip install --quiet -U "python-lsp-server[all]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [159]:
## Setup logging
import logging
import os
from dotenv import load_dotenv

load_dotenv(override=True)
logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the format
    handlers=[logging.StreamHandler()]  # Output to the console
)

# Define Spotify URI type

In [199]:
from typing import List, NewType

# Define a custom URI type
SpotifyURI = NewType('SpotifyURI', str)

In [200]:
from typing import Dict, List, Any, Annotated
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage, ToolMessage
from langgraph.graph.message import add_messages


class State(TypedDict):
    """
    Represents the state of the conversation and Spotify information

    Attributes:
        playlists (List[str]): A list of Spotify Playlists.
        tracks (List[str]): Track list for a Spotify Playlist
        new_playlist: (str) : New Spotify playlist data
        new_tracks: (List[str]): Tracks for the new playlist
    """
    new_playlist: str
    new_tracks: List[str]
    playlists: List[str]
    tracks: List[str]
    messages: Annotated[List[BaseMessage], add_messages]


# Define Spotify Tools

In [203]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from langchain_core.tools import tool
from typing import List, NewType


def get_spotify_client() -> spotipy.Spotify:
    """
    Initializes and returns a Spotify client with user authentication.

    Returns:
        spotipy.Spotify: An authenticated Spotify client.
    """
    auth_manager = SpotifyClientCredentials(
        client_id=os.environ.get("SPOTIFY_CLIENT_ID"),
        client_secret=os.environ.get("SPOTIFY_CLIENT_SECRET"),
    )
    return spotipy.Spotify(auth_manager=auth_manager)


@tool
def get_playlists() -> Dict[str, List[str]]:
    """
    Retrieves a list of the user's Spotify playlists.

    Returns:
        Dict[str, List[str]]: A dictionary containing a list of playlists.
    """
    sp = get_spotify_client()

    playlists = []
    try:
        playlists_raw = sp.current_user_playlists()
        while playlists_raw:
            for playlist in playlists_raw['items']:
                playlists.append(f"{playlist['id']}, {playlist['uri']}, {playlist['name']}")
            if playlists_raw['next']:
                playlists_raw = sp.next(playlists_raw)
            else:
                break
    except spotipy.SpotifyException as e:
        return {"error": str(e)}
    return {"playlists": playlists}

@tool
def get_track_list(playlist_id: str) -> Dict[str, List[Dict[str, Any]]]:
    """
    Get track list for a Spotify Playlist
    
    Args:
        playlist_id (str): A Spotify playlist ID 
    
    
    """
    sp = get_spotify_client()

    tracks = []
    try:
        tracks_raw = sp.playlist_tracks(playlist_id)
        while tracks_raw:
            for item in tracks_raw['items']:
                track = item['track']
                tracks.append({
                    'id': track['id'],
                    'name': track['name'],
                    'uri': track['uri'],
                    'artists': [artist['name'] for artist in track['artists']],
                    'album': track['album']['name']
                })
            if tracks_raw['next']:
                tracks_raw = sp.next(tracks_raw)
            else:
                break
    except spotipy.SpotifyException as e:
        return {"error": str(e)}
    return {"tracks": tracks}


@tool
def create_spotify_playlist(name: str, description: str = "Agentic Playlist") -> Dict[str, Any]:
    """
    Creates a new playlist on Spotify.

    This function only creates the playlist; it does not add tracks.

    Args:
        name (str): Name of the playlist.
        description (str, optional): Description of the playlist.

    Returns:
        Dict[str, Any]: A dictionary containing the new playlist's details.
    """
    sp = get_spotify_client()
    try:
        user_id = sp.current_user()['id']
        new_playlist = sp.user_playlist_create(
            user=user_id,
            name=name,
            public=True,
            description=description
        )
    except spotipy.SpotifyException as e:
        return {"error": str(e)}
    return {"new_playlist": new_playlist}

@tool
def add_tracks_to_playlist(playlist_id: str, tracks: List[SpotifyURI]) -> Dict[str, Any]:
    """
    Adds tracks to a Spotify playlist.

    Args:
        playlist_id (str): ID of the playlist.
        tracks (List[SpotifyURI]): List of Spotify track URIs.

    Returns:
        Dict[str, Any]: A dictionary indicating success or error.
    """
    sp = get_spotify_client()
    try:
        sp.playlist_add_items(playlist_id=playlist_id, items=tracks)
    except spotipy.SpotifyException as e:
        return {"error": str(e)}
    return {"success": True} 



# Define Search Tool

In [204]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=5,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
)
name = search_tool.get_name()
desc = search_tool.description
desc

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

# Create ToolNode

In [ ]:
from langgraph.prebuilt import ToolNode

tools = [get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist, search_tool]
tool_node = ToolNode(tools)

# Manual Test of Tool Infra

In [164]:
from langchain_core.messages import AIMessage, HumanMessage
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "get_playlists",
            "args" : {},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ],
)

tool_node.invoke({"messages": [message_with_single_tool_call]})

{'messages': [ToolMessage(content='{"playlists": ["1, spotify:playlist:2dbYK5b7J0F7IdH5n1TEUK,  RPreacher", "2, spotify:playlist:4ack9YtUhdxRayJDAqlfQe,  RP Bossa Nova Chill ", "3, spotify:playlist:75NW18NgdZuZeifrcjxKlZ,  GVF", "4, spotify:playlist:5iyONtUO21O88xw8pBblwh,  Now And Then", "5, spotify:playlist:4SDSUMg2HJcGFmHlupsU7U,  Jazz classics", "6, spotify:playlist:5fBxLG2wLAPVpx83402rHl,  Acdc", "7, spotify:playlist:17Yxm5hQbkbZwDcCTuiDXC,  einaudi", "8, spotify:playlist:2X6GccWfIqPWO6VYx405nO,  Eric Clapton Blues", "9, spotify:playlist:3smVXe8y8w8nt4lFH0HBkW,  Alchemy: Dire Straits", "10, spotify:playlist:2ifRZBXnX7UmIPUlGet5WZ,  Blues", "11, spotify:playlist:5nn2ePtd9gYEEDL3XnACCB,  brazil", "12, spotify:playlist:6YAntoKNXuogPKdV26b6XE,  CCR", "13, spotify:playlist:2VHkZvOMCkLF3B7TsKerKD,  Chill", "14, spotify:playlist:508lbLx9XUkHv5Xb45hxTP,  Classic Rock", "15, spotify:playlist:7zzDMq1sPwkZPYeqOQJ9qS,  The New Four Seasons - Vivaldi Recomposed", "16, spotify:playlist:4l8WcaYl

# Bind Tools to Model

In [209]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"))
llm_with_tools = llm.bind_tools(tools)

# Define Prompt

In [216]:
prompt = """
Create a step-by-step plan with atomic tasks to solve the following problem:

Build a Spotify Playlist with tracks of the same vibe as my 'New Rock' playlist. Here are some rules

1 - Only suggest tracks from artist not present on the 'New Rock' playlist
2 - Artist should have achieved success after year 2010
3 - Do not use 'The Raconteurs' as an artist in the new playlist
4 - Playlist should have around 100 tracks
5 - Arrange the tracks to create a smooth listening experience, considering tempo, energy, and mood.
6 - Add some hidden gems
"""

# First Message

In [217]:
from langchain_core.messages import AIMessage, HumanMessage

human_message = HumanMessage(prompt)
ai_tool_call_message = llm_with_tools.invoke([human_message])
#print(f"tool_call_message: {tool_call_message.pretty_print()}")


2024-11-11 11:05:16,896 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [218]:
print(ai_tool_call_message.content)

To build a Spotify playlist with tracks of the same vibe as your 'New Rock' playlist, while adhering to the rules provided, here's a step-by-step plan with atomic tasks:

### Step 1: Gather Information
1. **Retrieve 'New Rock' Playlist ID**: Identify the Spotify ID of your 'New Rock' playlist.
2. **Get Track List**: Obtain the list of tracks and corresponding artists present in the 'New Rock' playlist using the playlist ID.

### Step 2: Identify Suitable Artists
3. **Filter Out Artists**: From the list, eliminate 'The Raconteurs' and note the remaining artists to avoid.
4. **Search for New Artists**: Identify artists who achieved success after 2010 and are not in the 'New Rock' playlist.

### Step 3: Track Selection
5. **Discover Similar Tracks**: For each new artist, search for tracks that match the vibe of the 'New Rock' playlist.
6. **Select Hidden Gems**: Identify lesser-known tracks that fit the playlist's mood to include as hidden gems.

### Step 4: Create Playlist
7. **Compile T

# chat_prompt_template holds all the messages (Human, AI, Tool)

In [219]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt_template: ChatPromptTemplate = human_message + ai_tool_call_message
chat_prompt_template.format_messages()

[HumanMessage(content="\nCreate a step-by-step plan with atomic tasks to solve the following problem:\n\nBuild a Spotify Playlist with tracks of the same vibe as my 'New Rock' playlist. Here are some rules\n\n1 - Only suggest tracks from artist not present on the 'New Rock' playlist\n2 - Artist should have achieved success after year 2010\n3 - Do not use 'The Raconteurs' as an artist in the new playlist\n4 - Playlist should have around 100 tracks\n5 - Arrange the tracks to create a smooth listening experience, considering tempo, energy, and mood.\n6 - Add some hidden gems\n", additional_kwargs={}, response_metadata={}),
 AIMessage(content="To build a Spotify playlist with tracks of the same vibe as your 'New Rock' playlist, while adhering to the rules provided, here's a step-by-step plan with atomic tasks:\n\n### Step 1: Gather Information\n1. **Retrieve 'New Rock' Playlist ID**: Identify the Spotify ID of your 'New Rock' playlist.\n2. **Get Track List**: Obtain the list of tracks and 

# Tool Call (get_playlists)

In [169]:
import json
from langchain_core.messages import ToolMessage

playlists = tool_node.invoke({"messages": [ai_tool_call_message]})
response = playlists["messages"]
playlists: ToolMessage = response[0]
# We recreate the ToolMessage to take care of UNicode characters
tool_message = ToolMessage(content=json.dumps(json.loads(playlists.content)), name=playlists.name, tool_call_id=playlists.tool_call_id)
chat_prompt_template += tool_message


# Format Message List for LLM

In [170]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt_template.format_messages()

[HumanMessage(content="\nBuild a Spotify Playlist with tracks of the same vibe as my 'New Rock' playlist. Here are some rules\n\n1 - Only suggest tracks from artist not present on the 'New Rock' playlist\n2 - Artist should have achieved success after year 2010\n3 - Do not use 'The Raconteurs' as an artist\n4 - Playlist should have around 100 tracks\n5 - Feel free to add some hidden gems\n", additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qMkFqJ5eP6s5pZ9VdtCM2ZOw', 'function': {'arguments': '{}', 'name': 'get_playlists'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 222, 'total_tokens': 233, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 

# Send Second Tool Results to LLM

In [171]:
ai_tool_call_message_2 = llm_with_tools.invoke(chat_prompt_template.format_messages())
chat_prompt_template += ai_tool_call_message_2
chat_prompt_template.format_messages()


2024-11-11 02:07:52,707 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[HumanMessage(content="\nBuild a Spotify Playlist with tracks of the same vibe as my 'New Rock' playlist. Here are some rules\n\n1 - Only suggest tracks from artist not present on the 'New Rock' playlist\n2 - Artist should have achieved success after year 2010\n3 - Do not use 'The Raconteurs' as an artist\n4 - Playlist should have around 100 tracks\n5 - Feel free to add some hidden gems\n", additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qMkFqJ5eP6s5pZ9VdtCM2ZOw', 'function': {'arguments': '{}', 'name': 'get_playlists'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 222, 'total_tokens': 233, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 

# Tool Call (get_track_list)

In [172]:
response = tool_node.invoke({"messages": [ai_tool_call_message_2]})
response

{'messages': [ToolMessage(content='{"tracks": {"href": "https://api.spotify.com/v1/playlists/01tk0aitEuGK0ajWCkzdKc/tracks?offset=0&limit=100&additional_types=track", "items": [{"added_at": "2022-02-04T01:28:50Z", "added_by": {"external_urls": {"spotify": "https://open.spotify.com/user/n3pjz5onrh6nbg4xahe7dqryt"}, "href": "https://api.spotify.com/v1/users/n3pjz5onrh6nbg4xahe7dqryt", "id": "n3pjz5onrh6nbg4xahe7dqryt", "type": "user", "uri": "spotify:user:n3pjz5onrh6nbg4xahe7dqryt"}, "is_local": false, "primary_color": null, "track": {"preview_url": "https://p.scdn.co/mp3-preview/058a26ea3eb73c1a24145bae23d8cad4625a285a?cid=73dde5f5fcc54ef999b16118c1703326", "available_markets": ["AR", "AU", "AT", "BE", "BO", "BR", "BG", "CA", "CL", "CO", "CR", "CY", "CZ", "DK", "DO", "DE", "EC", "EE", "SV", "FI", "FR", "GR", "GT", "HN", "HK", "HU", "IS", "IE", "IT", "LV", "LT", "LU", "MY", "MT", "MX", "NL", "NZ", "NI", "NO", "PA", "PY", "PE", "PH", "PL", "PT", "SG", "SK", "ES", "SE", "CH", "TW", "TR", "

# Decode AI Tool Call message

In [173]:
track_list_tool_message = response["messages"][0]

In [174]:
content = json.loads(track_list_tool_message.content)
tracks_raw = content["tracks"]["items"]   # full raw spotify tracks data

# Track List Model

In [175]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Track(BaseModel):
    """This class represents a single track"""
    artist: str = Field(..., description="The artist of the track")
    name: str = Field(..., description="The name of the track")

class Tracks(BaseModel):
    """This class contains the track list for a Spotify playlist"""
    tracks: Optional[List[Track]] = Field(None, description="A list of Track objects representing the playlist tracks")


In [176]:
# Display track name and artist in the format <trackname>:<artist>

track_list = []

for item in tracks_raw:
    track = item['track']
    track_name = track['name']
    artist_name = track['artists'][0]['name']
    track = Track(artist=artist_name, name=track_name)
    track_list.append(track)  # Taking only the first artist
    # print(f"{track_name}:{artist_name}")

tracks = Tracks(tracks=track_list)
# print(tracks.model_dump_json(indent=2))

# Create Track List ToolMessage for LLM

In [177]:
tool_message = ToolMessage(content=tracks.model_dump_json(indent=2), name=track_list_tool_message.name, tool_call_id=track_list_tool_message.tool_call_id)

# Add Tool message containing playlist tracks to list of messages

In [178]:
chat_prompt_template += tool_message

In [179]:
chat_prompt_template.format_messages()

[HumanMessage(content="\nBuild a Spotify Playlist with tracks of the same vibe as my 'New Rock' playlist. Here are some rules\n\n1 - Only suggest tracks from artist not present on the 'New Rock' playlist\n2 - Artist should have achieved success after year 2010\n3 - Do not use 'The Raconteurs' as an artist\n4 - Playlist should have around 100 tracks\n5 - Feel free to add some hidden gems\n", additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qMkFqJ5eP6s5pZ9VdtCM2ZOw', 'function': {'arguments': '{}', 'name': 'get_playlists'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 222, 'total_tokens': 233, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 

# Send Tool Result to LLM

In [180]:
llm_with_structured_output = llm.with_structured_output(schema=Tracks, method='json_schema')
response = llm_with_structured_output.invoke(chat_prompt_template.format_messages())
# response = llm_with_tools.invoke(chat_prompt_template.format_messages())
response

2024-11-11 02:08:15,426 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tracks(tracks=[Track(artist='Royal Blood', name='Figure It Out'), Track(artist='The Struts', name='Could Have Been Me'), Track(artist='Highly Suspect', name='My Name Is Human'), Track(artist='Nothing But Thieves', name='Amsterdam'), Track(artist='Rival Sons', name='Pressure and Time'), Track(artist='The Amazons', name='In My Mind'), Track(artist='Shame', name='Concrete'), Track(artist='Wolf Alice', name='Giant Peach'), Track(artist='IDLES', name='Danny Nedelko'), Track(artist='Fontaines D.C.', name='Boys in the Better Land'), Track(artist='Black Honey', name='Corrine'), Track(artist='The Vaccines', name="I Can't Quit"), Track(artist='Courtney Barnett', name='Pedestrian at Best'), Track(artist='Fidlar', name='40oz. On Repeat'), Track(artist='King Gizzard & The Lizard Wizard', name='Rattlesnake'), Track(artist='The Mysterines', name="Love's Not Enough"), Track(artist='The Nude Party', name='Chevrolet Van'), Track(artist='Greta Van Fleet', name='Safari Song'), Track(artist='Sam Fender', n

In [181]:
response.tracks

[Track(artist='Royal Blood', name='Figure It Out'),
 Track(artist='The Struts', name='Could Have Been Me'),
 Track(artist='Highly Suspect', name='My Name Is Human'),
 Track(artist='Nothing But Thieves', name='Amsterdam'),
 Track(artist='Rival Sons', name='Pressure and Time'),
 Track(artist='The Amazons', name='In My Mind'),
 Track(artist='Shame', name='Concrete'),
 Track(artist='Wolf Alice', name='Giant Peach'),
 Track(artist='IDLES', name='Danny Nedelko'),
 Track(artist='Fontaines D.C.', name='Boys in the Better Land'),
 Track(artist='Black Honey', name='Corrine'),
 Track(artist='The Vaccines', name="I Can't Quit"),
 Track(artist='Courtney Barnett', name='Pedestrian at Best'),
 Track(artist='Fidlar', name='40oz. On Repeat'),
 Track(artist='King Gizzard & The Lizard Wizard', name='Rattlesnake'),
 Track(artist='The Mysterines', name="Love's Not Enough"),
 Track(artist='The Nude Party', name='Chevrolet Van'),
 Track(artist='Greta Van Fleet', name='Safari Song'),
 Track(artist='Sam Fender

In [182]:
chat_prompt_template += HumanMessage("You have reused artists from the New Rock playlist, try again")
llm_with_structured_output = llm.with_structured_output(schema=Tracks, method='json_schema')
response = llm_with_structured_output.invoke(chat_prompt_template.format_messages())
response

2024-11-11 02:08:37,499 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tracks(tracks=[Track(artist='Royal Blood', name="Trouble's Coming"), Track(artist='IDLES', name='Danny Nedelko'), Track(artist='Fontaines D.C.', name='Boys in the Better Land'), Track(artist='The Amazons', name='Mother'), Track(artist='Black Midi', name='John L'), Track(artist='King Gizzard & The Lizard Wizard', name='Self-Immolate'), Track(artist='Wolf Alice', name='Smile'), Track(artist='Biffy Clyro', name='Space'), Track(artist='The War on Drugs', name='Pain'), Track(artist='The Mysterines', name='In My Head'), Track(artist='Shame', name='Concrete'), Track(artist='Viagra Boys', name='Sports'), Track(artist='Sleaford Mods', name='Nudge It (feat. Amy Taylor)'), Track(artist='Parquet Courts', name='Wide Awake'), Track(artist='Courtney Barnett', name='Pedestrian at Best'), Track(artist='Tash Sultana', name='Jungle'), Track(artist='The Nude Party', name='Chevrolet Van'), Track(artist='Phoebe Bridgers', name='Kyoto'), Track(artist='Sam Fender', name='Seventeen Going Under'), Track(artist=

In [184]:
response.tracks

[Track(artist='Royal Blood', name="Trouble's Coming"),
 Track(artist='IDLES', name='Danny Nedelko'),
 Track(artist='Fontaines D.C.', name='Boys in the Better Land'),
 Track(artist='The Amazons', name='Mother'),
 Track(artist='Black Midi', name='John L'),
 Track(artist='King Gizzard & The Lizard Wizard', name='Self-Immolate'),
 Track(artist='Wolf Alice', name='Smile'),
 Track(artist='Biffy Clyro', name='Space'),
 Track(artist='The War on Drugs', name='Pain'),
 Track(artist='The Mysterines', name='In My Head'),
 Track(artist='Shame', name='Concrete'),
 Track(artist='Viagra Boys', name='Sports'),
 Track(artist='Sleaford Mods', name='Nudge It (feat. Amy Taylor)'),
 Track(artist='Parquet Courts', name='Wide Awake'),
 Track(artist='Courtney Barnett', name='Pedestrian at Best'),
 Track(artist='Tash Sultana', name='Jungle'),
 Track(artist='The Nude Party', name='Chevrolet Van'),
 Track(artist='Phoebe Bridgers', name='Kyoto'),
 Track(artist='Sam Fender', name='Seventeen Going Under'),
 Track(ar